In [2]:
import pandas as pd
import numpy as np

In [31]:
# data from 2014 April to 2014 July

In [2]:
April = pd.read_csv('uber-raw-data-apr14.csv')
May = pd.read_csv('uber-raw-data-may14.csv')
June = pd.read_csv('uber-raw-data-jun14.csv')
#July = pd.read_csv('uber-raw-data-jul14.csv')

In [3]:
April

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512
...,...,...,...,...
564511,4/30/2014 23:22:00,40.7640,-73.9744,B02764
564512,4/30/2014 23:26:00,40.7629,-73.9672,B02764
564513,4/30/2014 23:31:00,40.7443,-73.9889,B02764
564514,4/30/2014 23:32:00,40.6756,-73.9405,B02764


In [4]:
weather = pd.read_excel('NY Weather Data - Apr 14 - Jul 14.xlsx')  
weather

,2014,Temp. (°C),Dew Point (°C),Humidity (%),Sea Level Press. (hPa),Visibility (km),Wind (km/h),Precip. (mm),Events,Unnamed: 9,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,Apr,high,avg,low,high,avg,low,high,avg,low,...,avg,low,high,avg,low,high,avg,high,sum,NaN
1,1,14,8,3,2,-2,-4,76,56,35,...,1019,1016,16,16,16,26,16,40,0,NaN
2,2,9,6,3,5,3,1,92,81,70,...,1020,1019,16,16,16,16,6,32,0.76,Rain
3,3,17,11,4,5,1,-6,92,57,22,...,1019,1017,16,16,13,21,9,26,2.03,Rain
4,4,7,6,5,6,3,2,92,84,76,...,1015,1007,16,13,4,42,22,52,2.54,Rain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,27,29,26,22,22,21,19,90,78,65,...,1008,1005,16,16,16,35,19,40,0.25,Rain
125,28,29,25,21,22,18,14,90,67,44,...,1002,1000,16,15,10,50,26,63,12.95,"Rain , Thunderstorm"
126,29,26,22,18,13,11,8,68,50,32,...,1011,1006,16,16,16,34,17,45,0,NaN
127,30,27,22,18,17,12,9,73,54,35,...,1016,1015,16,16,16,32,15,37,0,NaN


In [5]:
apr_wea = weather.iloc[0:31].rename(columns={2014: "date", "Unnamed: 20": "weather"})[['date','weather']].fillna('Not Rain').drop([0])
may_wea = weather.iloc[31:64].rename(columns={2014: "date", "Unnamed: 20": "weather"})[['date','weather']].fillna('Not Rain').drop([31,32])
jun_wea = weather.iloc[64:96].rename(columns={2014: "date", "Unnamed: 20": "weather"})[['date','weather']].fillna('Not Rain').drop([64,65])
#jul_wea = weather.iloc[96:].rename(columns={2014: "date", "Unnamed: 20": "weather"})[['date','weather']].fillna('Not Rain').drop([96,97])

In [6]:
apr_wea['month'] = 4
may_wea['month'] = 5
jun_wea['month'] = 6
#jul_wea['month'] = 7

In [7]:
April['month'] = pd.to_datetime(April['Date/Time']).dt.month
April['date'] = pd.to_datetime(April['Date/Time']).dt.day

In [9]:
def merge_data(Mth,wea):
    Mth['month'] = pd.to_datetime(Mth['Date/Time']).dt.month
    Mth['date'] = pd.to_datetime(Mth['Date/Time']).dt.day
    final = Mth.merge(wea, how = 'left').drop(columns=['month', 'date'])
    return final

In [10]:
apr = merge_data(April,apr_wea)
may = merge_data(May,may_wea)
jun = merge_data(June,jun_wea)
#jul = merge_data(July,jul_wea)

In [11]:
df = apr.merge(may,how='outer').merge(jun,how='outer')
df.head()

,Date/Time,Lat,Lon,Base,weather
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512,Not Rain
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512,Not Rain
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512,Not Rain
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512,Not Rain
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512,Not Rain


In [12]:
df['weather_up']=df['weather']
df.loc[df['weather'] =='Fog','weather_up'] = 'Not Rain'
df.loc[df['weather'] !='Not Rain','weather_up'] = 'Rain'

In [13]:
df['weather_up'].value_counts()

Not Rain    1010336
Rain         870459
Name: weather_up, dtype: int64

In [14]:
df['Date/Time'] = pd.to_datetime(df['Date/Time'])

In [18]:
#B02598   Hinter, LLC.(Uber)
#B02682  Schmecken
#B02617  Weiter
#B02512  Unter
#B02764  Danach-NY

In [19]:
df['Base_Name']=df['Base']
df.loc[df['Base'] =='B02598','Base_Name'] = 'Hinter'
df.loc[df['Base'] =='B02682','Base_Name'] = 'Schmecken'
df.loc[df['Base'] =='B02617','Base_Name'] = 'Weiter'
df.loc[df['Base'] =='B02512','Base_Name'] = 'Unter'
df.loc[df['Base'] =='B02764','Base_Name'] = 'Danach-NY'

In [23]:
df = df.drop(columns='weather')
df = df.rename(columns={'weather_up':'Weather'})

,Date/Time,Lat,Lon,Base,Weather,Base_Name
0,2014-04-01 00:11:00,40.7690,-73.9549,B02512,Not Rain,Unter
1,2014-04-01 00:17:00,40.7267,-74.0345,B02512,Not Rain,Unter
2,2014-04-01 00:21:00,40.7316,-73.9873,B02512,Not Rain,Unter
3,2014-04-01 00:28:00,40.7588,-73.9776,B02512,Not Rain,Unter
4,2014-04-01 00:33:00,40.7594,-73.9722,B02512,Not Rain,Unter
...,...,...,...,...,...,...
1880790,2014-06-30 22:40:00,40.7332,-73.9872,B02764,Not Rain,Danach-NY
1880791,2014-06-30 23:12:00,40.7905,-73.9796,B02764,Not Rain,Danach-NY
1880792,2014-06-30 23:13:00,40.7640,-73.9887,B02764,Not Rain,Danach-NY
1880793,2014-06-30 23:15:00,40.7262,-73.9944,B02764,Not Rain,Danach-NY


In [24]:
df.head()

,Date/Time,Lat,Lon,Base,weather_up,Base_Name
0,2014-04-01 00:11:00,40.7690,-73.9549,B02512,Not Rain,Unter
1,2014-04-01 00:17:00,40.7267,-74.0345,B02512,Not Rain,Unter
2,2014-04-01 00:21:00,40.7316,-73.9873,B02512,Not Rain,Unter
3,2014-04-01 00:28:00,40.7588,-73.9776,B02512,Not Rain,Unter
4,2014-04-01 00:33:00,40.7594,-73.9722,B02512,Not Rain,Unter


In [25]:
#df.to_csv('uber_trip_data_14.csv')

In [3]:
uber_data = pd.read_csv('uber-raw-data-janjune-15.csv')
location = pd.read_csv('taxi+_zone_lookup.csv')

In [4]:
uber_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14270479 entries, 0 to 14270478
Data columns (total 4 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   Dispatching_base_num  object
 1   Pickup_date           object
 2   Affiliated_base_num   object
 3   locationID            int64 
dtypes: int64(1), object(3)
memory usage: 435.5+ MB


In [5]:
location

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NV,NaN


In [6]:
uber_data_up = uber_data.merge(location, how = 'left', left_on = 'locationID', right_on='LocationID').drop(columns= ['locationID','service_zone'])

In [8]:
uber_data_up['Base_Name']=uber_data_up['Dispatching_base_num']
uber_data_up.loc[uber_data_up['Dispatching_base_num'] =='B02598','Base_Name'] = 'Hinter'
uber_data_up.loc[uber_data_up['Dispatching_base_num'] =='B02682','Base_Name'] = 'Schmecken'
uber_data_up.loc[uber_data_up['Dispatching_base_num'] =='B02617','Base_Name'] = 'Weiter'
uber_data_up.loc[uber_data_up['Dispatching_base_num'] =='B02512','Base_Name'] = 'Unter'
uber_data_up.loc[uber_data_up['Dispatching_base_num'] =='B02764','Base_Name'] = 'Danach-NY'
uber_data_up.loc[uber_data_up['Dispatching_base_num'] =='B02765','Base_Name'] = 'Grun'
uber_data_up.loc[uber_data_up['Dispatching_base_num'] =='B02835','Base_Name'] = 'Dreist'
uber_data_up.loc[uber_data_up['Dispatching_base_num'] =='B02836','Base_Name'] = 'Drinnen'

In [9]:
uber_data_up = uber_data_up.drop(columns=['Affiliated_base_num'])

In [10]:
uber_data_up['month'] = pd.to_datetime(uber_data_up['Pickup_date']).dt.month

In [12]:
uber_data_up = uber_data_up[uber_data_up['month'] != 1]
uber_data_up = uber_data_up[uber_data_up['month'] != 2]
uber_data_up = uber_data_up[uber_data_up['month'] != 3]

In [14]:
uber_data_up['month'].value_counts()

6    2816895
5    2695553
4    2280837
Name: month, dtype: int64

In [15]:
uber_data_up = uber_data_up.drop(columns=['month'])

In [16]:
uber_data_up.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7793285 entries, 0 to 14270478
Data columns (total 6 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   Dispatching_base_num  object
 1   Pickup_date           object
 2   LocationID            int64 
 3   Borough               object
 4   Zone                  object
 5   Base_Name             object
dtypes: int64(1), object(5)
memory usage: 416.2+ MB


In [17]:
uber_data_up

,Dispatching_base_num,Pickup_date,LocationID,Borough,Zone,Base_Name
0,B02617,2015-05-17 09:47:00,141,Manhattan,Lenox Hill West,Weiter
1,B02617,2015-05-17 09:47:00,65,Brooklyn,Downtown Brooklyn/MetroTech,Weiter
2,B02617,2015-05-17 09:47:00,100,Manhattan,Garment District,Weiter
3,B02617,2015-05-17 09:47:00,80,Brooklyn,East Williamsburg,Weiter
4,B02617,2015-05-17 09:47:00,90,Manhattan,Flatiron,Weiter
...,...,...,...,...,...,...
14270474,B02765,2015-05-08 15:43:00,186,Manhattan,Penn Station/Madison Sq West,Grun
14270475,B02765,2015-05-08 15:43:00,263,Manhattan,Yorkville West,Grun
14270476,B02765,2015-05-08 15:43:00,90,Manhattan,Flatiron,Grun
14270477,B02765,2015-05-08 15:44:00,45,Manhattan,Chinatown,Grun


In [29]:
#uber_data_up.to_csv('uber_trip_data_15.csv')

In [56]:
total_trips = pd.concat([df['Date/Time'],uber_data_up['Pickup_date']])

In [61]:
total_trips = pd.DataFrame(data=total_trips,columns=['trips'])

In [63]:
total_trips.head()

,trips
0,2014-04-01 00:11:00
1,2014-04-01 00:17:00
2,2014-04-01 00:21:00
3,2014-04-01 00:28:00
4,2014-04-01 00:33:00


In [64]:
#total_trips.to_csv('total_trips.csv')